In [119]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

from langdetect import detect

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

Below code is inspired by: https://www.kaggle.com/brittabettendorf/nlp-on-airbnb-data/notebook

DATA VIEWING

In [92]:
#read the 'reviews_summary.csv' file with data
df_reviews_summary = pd.read_csv('data/reviews_summary.csv')
df_reviews_summary.shape

(401963, 6)

In [93]:
#checking the duplicates
df_reviews_summary.duplicated().sum()

0

In [94]:
df_reviews_summary.sample(20)

,listing_id,id,date,reviewer_id,reviewer_name,comments
20042,252993,180130245,2017-08-09,20468108,Thomas,Alles super
204333,9273945,314084655,2018-08-26,14009368,Lavandier,Emplacement super \nAccueil super \nLogement atypique
186813,7872740,340273185,2018-10-23,178846158,Rebecca,schöne und gemütliche wohnung;wir haben uns sehr wohl gefühlt!
343892,20274962,324597090,2018-09-17,114792247,Arne,"Geschichtsträchtige, tolle Unterkunft,gemütliches Bett,Super Gastgeber.Wer am Abend oder in der Nacht dem Hauptstadttrubel etwas entkommen will ist hier genau richtig aufgehoben.Verkehrsgünstig gelegen und gemütlich."
317146,18176485,181202349,2017-08-12,144730538,Daniel,"Fantastic location (close to nightclub district, three train stops away from alexanderplatz), very comfortable bed, big room, clean kitchen and bathroom. Highly recommended."
373706,23073954,339093051,2018-10-21,21431014,Anja,Super praktische sehr zentrale Unterkunft zu vernünftigem Preis. Flexible Gastgeber. Betten je max. 1.40
58377,969200,83924273,2016-07-04,71537704,Rob,Alan & Kaisa were fantastic. Alan went out of his way to make myself feel comfortable with not only the listing but the area and what to do and where to go. wonderful reception & home. thanks guys.
87350,1867179,134418928,2017-02-27,7940365,Cindy,"My family and I enjoyed our stay in Prenzlauerberg. I lived in Berlin years ago, but never had the opportunity to explore the East. This flat is ideally situated to use the S-bahn or tram (""M"") system. I heartily recommend downloading the BVG app on your phone to help you quickly navigate Berlin on the fly :-) And purchase your ABC card right at Tegel for the week (or however long you need) which was cost and time efficient for us! If you arrive at Tegel, take the TXL bus to Buesselstr a..."
293057,16451952,196501692,2017-09-23,46836314,Stefan,"Schöne kleine Wohnung in Neukölln, gut für bis zu zwei Personen. Gute Ausstattung - Alles da, was man braucht. Leider sehr laut, da an verkehrsreicher Einmündung im EG gelegen. Schlafen nur mit Gehörschutz möglich."
331248,19267130,248347516,2018-03-31,88625031,Federico,"I strongly suggest this flat not only for its central position close to Alexanderplatz but especially for the two amazing hosts that live in it. I had an amazing time, they are clean, friendly and will make your staying in Berlin special"


In [95]:
#read the 'listings_summary.csv' file with data
df_listings_summary = pd.read_csv('data/listings_summary.csv')
df_listings_summary.shape

(22552, 96)

In [96]:
df_listings_summary.sample(10)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
2931,4442111,https://www.airbnb.com/rooms/4442111,20181107122246,2018-11-07,Neukölln aus der Vogelperspektive,"Bei uns hast du den Überblick über den ganzen Schillerkiez. Am Herrfurthplatz gelegen, 3 Min. zum Tempelhofer Feld","Von unserem Süd - Balkon aus, hast Du den vollen Tag Sonne und den ganzen Schillerkiez im Blick. Die Stimmung ist einzigartig, wenn abends die Menschen auf dem Herrfurthplatz Gitarre spielen und das Leben genießen. Werde Teil davon und erlebe unsere Stadt Berlin in ihrer wundervollen Lebendigkeit.","Bei uns hast du den Überblick über den ganzen Schillerkiez. Am Herrfurthplatz gelegen, 3 Min. zum Tempelhofer Feld Von unserem Süd - Balkon aus, hast Du den vollen Tag Sonne und den ganzen Schillerkiez im Blick. Die Stimmung ist einzigartig, wenn abends die Menschen auf dem Herrfurthplatz Gitarre spielen und das Leben genießen. Werde Teil davon und erlebe unsere Stadt Berlin in ihrer wundervollen Lebendigkeit. Wir haben einen Waschtrockner, Kühlschrank, Herd, Spülmaschine, Badewanne, Int...",none,"Neukölln ist derzeit der lebendigste und aufregenste Stadtteil Berlins. Immer mehr Cafés, Galerien und Restaurants eröffnen. Das besondere am Schillerkiez ist die direkte Nähe zu zwei Parks, der Hasenheide und dem atemberaubenden stillgelegten Flughafen Tempelhof. Keine andere Stadt in Europa hat soviel Freifläche mitten in der Stadt. Es konnte eine Bebauung durch des Feldes verhindert werden und immer mehr Menschen nutzen den Wind und die Weite für allerlei sportliche Aktivität: Spazie...",Nicht die Sonnencreme vergessen. ;-),Direkter Anschluss zu U-Bahnen und Bussen. U-Bahnhof Boddinstrasse Die U8 fährt direkt zum Alexanderplatz. Zwei Stationen weiter hat man an der Hermannstr. Anbindung zu S- und Ringbahnen.,"Wir haben einen Waschtrockner, Kühlschrank, Herd, Spülmaschine, Badewanne, Internet.","Wir werden in den Zeiten ebenfalls verreist sein. Es kann sich aber jederzeit an das Café LUX im Haus gewandt werden. Wir haben auch sehr liebenswerte Nachbarn direkt nebenan, bei denen man jederzeit klingeln kann, wenn man mal nicht weiter weiß.","Unsere Wohnung ist uns lieb und teuer. Fühl dich wie Zuhause und gehe anständig mit unserem Heim um. :) Im Sommer wäre es großartig, wenn du die Blumen auf dem Balkon gießen könntest. We love our apartment. Here is, where we live.It is really important for us. We hope, you'll enjoy it the way we do. Please take care of it and be careful with everything inside. In summer times, it would be phenomenal, if you could watering the flours. The ones on the balcony too.

In [97]:
# merging full reviews + add only specific columns from listings dataframe
df = pd.merge(df_reviews_summary, df_listings_summary[['neighbourhood_group_cleansed', 'host_id', 'latitude',
                          'longitude', 'number_of_reviews', 'id', 'property_type']], 
              left_on='listing_id', right_on='id', how='left')

df.rename(columns = {'id_x':'id', 'neighbourhood_group_cleansed':'neighbourhood_group'}, inplace=True)
df.drop(['id_y'], axis=1, inplace=True)

In [98]:
df.sample(10)

,listing_id,id,date,reviewer_id,reviewer_name,comments,neighbourhood_group,host_id,latitude,longitude,number_of_reviews,property_type
250082,13320037,151849242,2017-05-14,10036068,Paulina,"Sven's apartment is amazing, comfortable, spacious and clean. Located 10 min on foot from Hauptbahnhof (Central Station). Tram and metro stations are just few minutes away. I recommend it to everyone.",Mitte,75327580,52.528962,13.380686,155,Apartment
21058,257658,9056351,2013-12-03,1773435,Asun,"Mi estancia en casa de Boris y Tatiana ha sido muy agradable. Son una pareja entrańable. Ambos conocen perfectamente Berlín y me han dado valiosos consejos. La casa es exactamente como Boris la describía en el anuncio Airbnb. Es amplia, luminosa, cómoda y realmente bien localizada. La organización de la web ha sido perfecta. Sin duda, repetiré experiencia en otra ciudad y lo recomendaré a mis amigos.",Pankow,1356042,52.530252,13.410557,201,Apartment
120213,3515365,19953073,2014-09-21,1592231,Edwin,"My stay at this place was amazing. From the beginning, we had a lot of communication which was great. They even welcome me with a dessert. \r\n\r\nThe space is huge and everything (bathroom, wifi, kitchen, etc) works perfectly. \r\n\r\nThe neighbourhood is very calm and walkable, with many restaurants close and stores.",Mitte,8579279,52.529204,13.344330,10,Apartment
242342,12663936,159508672,2017-06-11,23318255,Hui,Er geht aber die Straße ist zu laut und das Zimmer nicht so sauber und hell wie das Bild,Tempelhof - Schöneberg,68802793,52.496718,13.346931,29,Bed and breakfast
43587,697887,38128694,2015-07-13,2098553,Christian,Cool and spacious hipster apartment in a nice area. Be prepared to noise from the trains which almost runs through the bedroom.,Pankow,3576166,52.549810,13.407016,197,Loft
316321,18135638,185282690,2017-08-21,66547925,Jeremy,Ich würde diese schöne Wohnung immer wieder Buchen bzw. weiterempfehlen. Stefan ist ein super Gastgeber. Vielen lieben Dank!,Friedrichshain-Kreuzberg,8352102,52.502958,13.372670,10,Apartment
290340,16283253,170738552,2017-07-16,3776064,Behzad,Nice place !,Friedrichshain-Kreuzberg,16913288,52.497711,13.423899,151,Apartment
203130,9193748,111067649,2016-10-30,29531422,Hanno,"Erreichbarkeit war gesichert bei Anreise. Alles sehr easy, unkompliziert.\r\nAbsolute Ruhe durch Hinterhoflage. Angenehme Umgebung, gute Mischung von Galerien und Gastro. Theater und Ausgehpossibilities.\r\nGute Anbindungen an ÖPVN.",Mitte,47556192,52.527589,13.391752,24,Apartment
124307,3823118,18145071,2014-08-22,11970618,Bianca,"Tolle, geräumige Wohnung, netter Vermieter!",Neukölln,19610820,52.468126,13.422397,93,Apartment
48924,784782,20249097,2014-09-26,20881032,Kanako,"I really relaxed at Alex's home. Lovey and very clean room and bathroom. I wish I could stay much longer. Also the location is very nice. From the bus stop which is in the front of the apartment, #200 bus takes you directly to the centre of Berlin.",Mitte,4137814,52.507547,13.345073,188,Condominium


In [99]:
#group by host_id and count the number of flats for each host 
flats_per_host = pd.DataFrame(df.groupby('host_id')['listing_id'].nunique())
#top 20 sorted
flats_per_host.sort_values(by=['listing_id'], ascending=False, inplace=True)
flats_per_host.head(20)

,listing_id
host_id,
1625771,44
8250486,38
2293972,28
59442407,19
87442687,16
54942921,16
8912086,15
34705166,15
79430806,15


In [100]:
def check_top_host_neighbourhood(df, host_id):
    top_host = df['host_id'] == host_id
    df[top_host]['neighbourhood_group'].value_counts()
    print ('Host number: '+ str(host_id))
    return pd.DataFrame(df[top_host].groupby('neighbourhood_group')['listing_id'].nunique())    

In [101]:
def check_top_host_property(df, host_id):
    top_host = df['host_id'] == host_id
    df[top_host]['neighbourhood_group'].value_counts()
    print ('Host number: '+ str(host_id))
    return pd.DataFrame(df[top_host].groupby('property_type')['listing_id'].nunique())

In [108]:
#checking the no. 1 host, he has 44 apartments in 8 different districts
check_top_host_neighbourhood(df, flats_per_host.index[0])

Host number: 1625771


,listing_id
neighbourhood_group,
Charlottenburg-Wilm.,3
Friedrichshain-Kreuzberg,2
Lichtenberg,1
Mitte,10
Neukölln,1
Pankow,19
Tempelhof - Schöneberg,7
Treptow - Köpenick,1


In [107]:
check_top_host_property(df, flats_per_host.index[0])

Host number: 1625771


,listing_id
property_type,
Apartment,44


In [109]:
#checking the no. 2 host, turned out to be a hostel
check_top_host_neighbourhood(df, flats_per_host.index[1])

Host number: 8250486


,listing_id
neighbourhood_group,
Friedrichshain-Kreuzberg,1
Mitte,37


In [110]:
check_top_host_property(df, flats_per_host.index[1])

Host number: 8250486


,listing_id
property_type,
Hostel,38


In [112]:
#checking the no. 3 host, different properties in two districts
check_top_host_neighbourhood(df, flats_per_host.index[2])

Host number: 2293972


,listing_id
neighbourhood_group,
Friedrichshain-Kreuzberg,24
Lichtenberg,4


In [113]:
check_top_host_property(df, flats_per_host.index[2])

Host number: 2293972


,listing_id
property_type,
Aparthotel,3
Apartment,11
Hostel,6
Loft,1
Serviced apartment,7


DATA PREPARATION

In [114]:
#missing values, 496 comments are missing
df.isna().sum()

listing_id               0
id                       0
date                     0
reviewer_id              0
reviewer_name            0
comments               496
neighbourhood_group      0
host_id                  0
latitude                 0
longitude                0
number_of_reviews        0
property_type            0
dtype: int64

In [115]:
#496 is not so much so I delete those data
df.dropna(inplace=True)
df.isna().sum()

listing_id             0
id                     0
date                   0
reviewer_id            0
reviewer_name          0
comments               0
neighbourhood_group    0
host_id                0
latitude               0
longitude              0
number_of_reviews      0
property_type          0
dtype: int64

LANGUAGE DETECTION

In [121]:
#function to detect language of comment
def language_detection(text):
    try:
        return detect(text)
    except:
        return None

In [122]:
%%time
df['language'] = df['comments'].apply(language_detection)

Wall time: 37min 44s


In [124]:
#write the dataframe to a csv file in order to avoid the long runtime
df.to_csv('prepared_df', index=False)
df = pd.read_csv('prepared_df')

In [126]:
df.sample(10)

,listing_id,id,date,reviewer_id,reviewer_name,comments,neighbourhood_group,host_id,latitude,longitude,number_of_reviews,property_type,language
44584,704306,325627047,2018-09-20,139676170,Daniel,"Traumhafte Unterkunft, unkomplizierter Ablauf, super Gastgeber! Immer wieder gerne:)! BG Daniel",Pankow,3616161,52.554239,13.414013,35,Apartment,de
238495,12202768,120264933,2016-12-09,41455178,Hadi,"Sehr ruhige Lage und sehr gemütlich. Jedem zu empfehlen und Paul ist auch, obwohl ich ihn persönlich nicht kennengelernt habe, sehr zuvorkommend. Er scheint auch einen ausgewachsenen Sinn für Humor zu haben, weil er einen meiner trockenen Witze lustig gefunden hat.",Mitte,47702685,52.530775,13.382649,147,Apartment,de
32987,485726,71639176,2016-04-27,9453115,Burton,"I didn't meet Alex but I met his friend Martin who was waiting for me when I arrived. He was a wonderful representative for Alex, and he explained everything to me.",Charlottenburg-Wilm.,2347025,52.499085,13.336407,73,Apartment,en
296903,16810170,193479767,2017-09-13,133508427,Donato,The host canceled this reservation 79 days before arrival. This is an automated posting.,Friedrichshain-Kreuzberg,82032528,52.521391,13.449764,19,Apartment,en
33293,493072,86541040,2016-07-16,50205933,Tarek,We had a great stay at Gregor's. He was very friendly and helpful (plenty of useful tips about where to go). The apartment itself is clean and is well located.,Mitte,1903768,52.529172,13.402894,168,Apartment,en
322459,18586882,312375697,2018-08-23,21458082,Ivica,"Für Künstler, Musiker, Tagediebe, und andere absolut zu empfehlen. Sehr gerne wieder. Daumen hoch. Schön, dass es so etwas im schicken Berlin noch gibt.",Pankow,4259960,52.540961,13.422738,135,Other,de
128505,3992720,124641322,2017-01-02,107329913,Bob,"Michaela's appartement was een goede uitvalbasis om de stad te verkennen en het nieuwe jaar te starten in Berlijn. We kwamen eerder aan dan verwacht maar dat was geen probleem. Michaela vertelde ons dat we gebruik konden maken van een lift in een aangrenzend gebouw. Deze bracht ons naar de 5e etage, hier konden we een gang door naar een ander gebouw waar nog een trap was die ons leidde naar het appartement. Michaela stond al in de deuropening en ontving ons hartelijk. Ze leidde ons rond en ...",Mitte,20697242,52.529161,13.346477,47,Apartment,nl
188367,7970986,46028208,2015-09-07,41678521,Ziv And Avigail,Michael was very kind and helpful. He was available to any question and problem. The apartment was clean and tidy and in a great location near the Hackecher Market.\n\nTruly Recommended!! \n\n,Mitte,42061066,52.525053,13.404417,6,Apartment,en
334568,19569434,184790665,2017-08-20,97280315,Lucie,Yousef was young and nice host. Flat was in good location in centrum and near underground. Room was good with comfortable bed.,Mitte,127387738,52.504903,13.345976,9,Apartment,en
372649,22989852,257455249,2018-04-26,60992567,Ayhan,"Der Aufenthalt war sehr angenehm. Die Unterkunft ist sauber und modern eingerichtet und hat alles was man braucht. Obwohl wir nur zu zweit waren ist die Unterkunft für bis zu 6 Personen optimal. Lage ist zentral. Haltestellen für den öffentlichen Nahverkehr in wenigen Minuten erreichbar. Der Kontakt war freundlich und unkompliziert. Wir werden auf jeden Fall wieder hier buchen, wenn wir noch mal nach Berlin kommen.",Pankow,155722828,52.535264,13.434414,62,Apartment,de
